# Se soluciona el bug a partir de una aproximación diferente al problema. Ya no se muta primero los diferentes si sobran dimensiones a mutar, entonces se mutan los iguales pero siempre muta el total de dimensiones otorgado por la posicion pos_temp; además todas las dimensiones se seleccionan de forma aleatoria

# Se programa la fase sólida.

# se arreglaron las matrizes de resultados para ser una sola

# Librerías

In [1]:
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)#Prueba evaluación y entrenamiento un solo individuo. 
#Vectores completos
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import random

In [2]:
import seaborn as sns

# Databases

In [3]:
#Reading all the cvs
#Nss learning
nsl_training=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Tra.csv')
#Nsl testing
nsl_testing=pd.read_csv('./Final_Datasets/MINMAX_FD_NSL_Test.csv')

# Labels

In [4]:
#NSL TESTING LABELS

#Index binary (2 clases)
L_nsl_testing_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_testing_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_testing_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_testing_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_testing_indextipeofattack.csv")


#nsl TRAINING LABELS
# 2 clases
L_nsl_training_indexbinary=pd.read_csv("./Final_Datasets/L_nsl_training_indexbinary.csv")
#Index by Group (5 clases)
L_nsl_training_indexbygroup=pd.read_csv("./Final_Datasets/L_nsl_training_indexbygroup.csv")
#Index tipe of attack (42? clases)
L_nsl_training_indextipeofattack=pd.read_csv("./Final_Datasets/L_nsl_training_indextipeofattack.csv")

# Conversión de Pandas->Numpy (array) not Matrix

In [5]:
nsl_training=nsl_training.as_matrix(columns=None)
nsl_testing=nsl_testing.as_matrix(columns=None)

# Funcion para calcular M dependiendo de las iteraciones

In [6]:
def M_dimensiones_a_mutar(Perror,i_actual,i_max,M_Max=40):
    iteraciones=1-(i_actual/i_max)
    M=M_Max*np.amin([iteraciones,Perror])
    M=np.array(np.round_(M))
    return M

# Funcion para calcular el rendimiento F(s)

In [7]:
def funcion_rendimiento(Perror_actual,unos,N=40,alfa=.3):
    cardinalidad=unos
    rendimiento=alfa*Perror_actual+(1-alfa)*(cardinalidad/N)
    return rendimiento

# Declaración del modelo

In [8]:
from sklearn.ensemble import RandomForestClassifier

modelo=RandomForestClassifier(criterion='entropy',n_jobs=2, 
                              bootstrap=True, max_depth=10, max_features='sqrt', min_samples_leaf=2, 
                              min_samples_split=100, n_estimators=8)


# Declarando e modelo para la distancia

In [9]:
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('jaccard')

# IMO

# Funciones

# Eval sencilla

In [10]:
#funcion para la evaluacion sencilla de un elemento
#la evaluación sencilla consta de 
#indices_uno_poblacion, guarda los indices de la poblacion de aniones o cationes las cuales contienen los
#parametros de 1
def evaluacion_sencilla(poblacion,matriz_de_evaluacion,indices_unos_poblacion,modelo):
    """Regresa la matriz dd resultados
    los indices dónde se encuentran los unos"""
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        #filtrado de la base de datos
        training_temp=poblacion[indice_elemento]*nsl_training
        testing_temp=poblacion[indice_elemento]*nsl_testing    
        #se entrena el modelo
        modelo.fit(training_temp,training_etiquetas.values.ravel())
        #se evalua el modelo
        predicciones_temp=modelo.predict(testing_temp)
        #se obtiene el accuracy score
        matriz_de_evaluacion[indice_elemento,0]=accuracy_score(testing_etiquetas,predicciones_temp)
        #se obtiene la probabilidad de error
        matriz_de_evaluacion[indice_elemento,1]=1-accuracy_score(testing_etiquetas,predicciones_temp)
        #se obtienen la cantidd de los indices de los unos (dimensinoes activas)
        matriz_de_evaluacion[indice_elemento,2]=np.array(np.where(poblacion[indice_elemento]==1)).shape[1]
        #se obtienen los indices de los unos
        indices_unos_poblacion.append(np.where(poblacion[indice_elemento]==1))
        
        print("unos temporales: %s" %M_E_A[indice_elemento,2])
    return [matriz_de_evaluacion,indices_unos_poblacion]

        

# Eval compuesta

In [11]:

#Matriz con resultados para su actualizacion
#0-column (distancia - Jaccard)
#1-column (fuerza)
#2-column (cantidad de dimensiones similares con el mejor)

#funcion para la evaluacion compuesta de un elemento
#la evaluación sencilla consta de 
def evaluacion_compuesta(poblacion,mejor_elemento_pob_contraria,matriz_de_evaluacion):
    """Regresa la matriz dd resultados compuestos, la cual incluye
    distancia (jacard), fuerza, y los indices de los vectores iguales entre 
    el elemento x y el mejor de la población"""
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        #se calcula el indice [elemento,3]->Distancia
        a=poblacion[indice_elemento]
        b=mejor_elemento_pob_contraria
        matriz_de_evaluacion[indice_elemento,3]=dist.pairwise([a,b])[0,1]
        #se calcula el indice [elemento,4]->fuerza
        x=matriz_de_evaluacion[indice_elemento,3]
        fuerza=(1/(1+(2.71828**(-0.1/x))))
        #se almacena la fuerza en la matriz
        matriz_de_evaluacion[indice_elemento,4]=fuerza
        #se calcula el rendimiento F(S)
        #se calcula el indice [elemento,5]->Rendimiento
        matriz_de_evaluacion[indice_elemento,5]=funcion_rendimiento(matriz_de_evaluacion[indice_elemento,1],
                                matriz_de_evaluacion[indice_elemento,2],N=40,alfa=.3)
                             
 
        print("indice_elemento %s" %indice_elemento)
    return [matriz_de_evaluacion]


# Fase líquida

In [12]:
def fase_liquida(poblacion,mejor_pob_contraria,matriz_resultados):
    """poblacion=poblacion de aniones/cationes
    mejor_pob_contraia=mejor elemento de la poblacion contraria a partir de la cual mutar
    fase=liquida,solida"""
    #probando la fase líquida
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        
        print("\n")
        print("Me encuentro trabajando en el elmento %s de %s totales" %(indice_elemento,poblacion.shape[0]))
        respaldo_temporal_elemento=np.array(poblacion[indice_elemento])
        print("Elemento:")
        print(respaldo_temporal_elemento)
        
        
        #Encuentra el total de dimensiones M posibles a mutar POR ELEMENTO
        M=M_dimensiones_a_mutar(matriz_resultados[indice_elemento,1],i_actual,i_max)
        
        print("Tiene una probabilidad de error de: %s" % matriz_resultados[indice_elemento,1])
        print("Para el elemento %s se mutan un total máximo de M=%s dimensiones" %(indice_elemento,M))
        
        #Encuentra el total de dimensinoes posibles a mutar, dependiendo del elemento (distancia,fuerza)
        #y que ademas depende de M
        #posicion_temp=cantidad de dimensinoes a mutar por elemento. 
        
        #matriz_resultados[elemento,5]->posicion
        #matriz_resultados[elemnto,4]->fuerza

        
        #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
        posicion_temp=matriz_resultados[indice_elemento,4]*M
        #se redondea para que se un total de dimensiones entero
        posicion_temp=int(np.round_(posicion_temp))
        
        print("Cantidad de dimensiones a mutar específicas para ese elemento: %s" %(posicion_temp))
     
        
        #se actualiza la matriz de resultados
        matriz_resultados[indice_elemento,6]=posicion_temp
        
        #muta unicamente los iguales, aún pese a que haya más elementos posibles a mutar M
        #debido a la posicion_temp
    
        a=np.where((poblacion[indice_elemento]==mejor_pob_contraria)==False)
        #para trabajar sin dificultades se convierte a una lista plana
        b=np.concatenate(a).ravel().tolist()
                
        print("El elemento mejor es: ")
        print(mejor_pob_contraria)
        print("El elemento a evaluar es: ")
        print(poblacion[indice_elemento])
        print("Los indices que son diferentes son %s" %(a))
        
        #se seleccionan posicion_temp elementos de forma ALEATORIA, para no elegirse siempre las primeras
        #dimensiones
        
        dimensiones_utilizadas=0
        criterio_fase_solida=0
        
        
        #--------------------------------
        
        if (len(b)!=0):
            if (posicion_temp>len(b)):
                #solución al problema de cuando hay más cantidad de 
                #dimensiones posibles que de elementos, es decir,
                #se toma todo el conjunto de dimensiones iguales
                print("Entre al primer if")
                diferentes_temp_indices=b
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                
                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

            else:
                
                print("Entre al primer else")
                #cuando hay más dimensiones diferentes que dimensiones posibles a mutar
                #se escogen de manera aleatoria
                c=random.sample(b,posicion_temp)
                diferentes_temp_indices=sorted(c)
                diferentes_temp_cantidad=len(diferentes_temp_indices)

                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                
            #while para mutar las dimensiones diferentes
            #se resta uno por el while, si son 5, toma 6, debido a que empieza en cero
            while((dimensiones_utilizadas<diferentes_temp_cantidad)):
                print("Entre al primer while")
                print("Diferentes temp cantidad %s" %diferentes_temp_cantidad)
                #muta los diferentea
                #print("\n\n")
                #print(diferentes_temp_indices)
                #print(dimensiones_utilizadas)
                #print(type(diferentes_temp_indices))
                #print(diferentes_temp_indices[0,dimensiones_utilizadas])
                
                print("indice_elemento: %s" %indice_elemento,"diferentes temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                if (poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] ==0):
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =1
                else:
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =0


                print("Se muto la dimension: %s" %(diferentes_temp_indices[dimensiones_utilizadas]))
                print("El elemento de la poblacion %s mutado es: ")
                print(poblacion[indice_elemento])

                dimensiones_utilizadas=dimensiones_utilizadas+1      
                
                
        else:
            #poblacion 
            poblacion[indice_elemento,7]=1
            #cuando son exactamente iguales el criterio para la fase sólida aumenta
            criterio_fase_solida=criterio_fase_solida+1
            
    
    return [poblacion,matriz_resultados,criterio_fase_solida]
            





In [13]:
#funcionmejor elemento de la poblacion en cuanto al menor rendimiento F(S)

In [14]:
def mejor_individuo(poblacion_matriz_evaluacion):
    mejor=np.argmin(poblacion_matriz_evaluacion[:,5])
    return mejor
    

# Declaracion de parámetros

In [15]:
#Declaracion de la poblacion de forma aleatoria

A=np.random.randint(2, size=(10,40))
C=np.random.randint(2, size=(10,40))


#Matriz Dimensiones activas (lista):
#dimensiones_activas_aniones=[]
#dimensiones_activas_cationes=[]

I_A=[]
I_C=[]

#Matriz de evaluacion con todos los parámetros
M_E_A=np.zeros([10,8])
M_E_C=np.zeros([10,8])



#Declaracion de las matrices para
#Matriz Resultados sencillos
#0-column (score)
#1-column (p-error
#2-column (dimensiones activas)


#Matriz con resultados para su actualizacion
#3-column (distancia - Jaccard)
#4-column (fuerza)


#5-column cantidad de dimensiones M a mutar
#6-column Cantidad de dimensiones M que cumplieron la condición de su fase sólida/líquida


#2-column (cantidad de dimensiones similares con el mejor)



#declaracion de etiquetas
training_etiquetas=L_nsl_training_indexbinary
testing_etiquetas=L_nsl_testing_indexbinary

[M_E_A,IA]=evaluacion_sencilla(A,M_E_A,I_A,modelo)


#mejor elemento de forma temporal 
mejor_elemento=np.random.randint(2, size=(40))
[M_E_A]=evaluacion_compuesta(A,mejor_elemento,M_E_A)




unos temporales: 21.0
unos temporales: 20.0
unos temporales: 15.0
unos temporales: 23.0
unos temporales: 17.0
unos temporales: 18.0
unos temporales: 24.0
unos temporales: 18.0
unos temporales: 15.0
unos temporales: 15.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9


In [16]:
M_E_A

array([[  0.78813822,   0.21186178,  21.        ,   0.63333333,
          0.53939185,   0.43105853,   0.        ,   0.        ],
       [  0.71028701,   0.28971299,  20.        ,   0.62068966,
          0.54019085,   0.4369139 ,   0.        ,   0.        ],
       [  0.75189638,   0.24810362,  15.        ,   0.79310345,
          0.53148002,   0.33693109,   0.        ,   0.        ],
       [  0.76147806,   0.23852194,  23.        ,   0.61290323,
          0.5406992 ,   0.47405658,   0.        ,   0.        ],
       [  0.74714989,   0.25285011,  17.        ,   0.72413793,
          0.53446903,   0.37335503,   0.        ,   0.        ],
       [  0.76019163,   0.23980837,  18.        ,   0.59259259,
          0.54208764,   0.38694251,   0.        ,   0.        ],
       [  0.74945659,   0.25054341,  24.        ,   0.66666667,
          0.53742982,   0.49516302,   0.        ,   0.        ],
       [  0.76387349,   0.23612651,  18.        ,   0.77419355,
          0.53224682,   0.3858379

In [17]:
# corregir la fase líquida para que seleccione de forma aleatoria los indices de los diferentes, ya que se
# tendería a escoger los primeros

In [18]:
mejor_elemento

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0])

In [22]:
i_actual=100
i_max=300
#A[0,:]=[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
#       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]
#A[0,:]=[1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
 #      0, 1, 0, 1, 2, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1]

criterio_fase_solida=0
[A,M_E_A,criterio_fase_solida]=fase_liquida(A,mejor_elemento,M_E_A)
[M_E_A,IA]=evaluacion_sencilla(A,M_E_A,I_A,modelo)
[M_E_A]=evaluacion_compuesta(A,mejor_elemento,M_E_A)




Me encuentro trabajando en el elmento 0 de 10 totales
Elemento:
[1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1
 0 1 0]
Tiene una probabilidad de error de: 0.28669653551
Para el elemento 0 se mutan un total máximo de M=11.0 dimensiones
Cantidad de dimensiones a mutar específicas para ese elemento: 7
El elemento mejor es: 
[1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1
 0 1 0]
El elemento a evaluar es: 
[1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1
 0 1 0]
Los indices que son diferentes son [12 14 25]
Entre al primer if
indice_elemento: 0 diferentes temp indices [12, 14, 25] dimensinoes utilizadas: 0
Entre al primer while
Diferentes temp cantidad 3
indice_elemento: 0 diferentes temp indices [12, 14, 25] dimensinoes utilizadas: 0
Se muto la dimension: 12
El elemento de la poblacion %s mutado es: 
[1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1
 0 1 0]
Entre al primer while
Dif

unos temporales: 20.0
unos temporales: 20.0
unos temporales: 18.0
unos temporales: 20.0
unos temporales: 20.0
unos temporales: 20.0
unos temporales: 20.0
unos temporales: 20.0
unos temporales: 20.0
unos temporales: 20.0
indice_elemento 0
indice_elemento 1
indice_elemento 2
indice_elemento 3
indice_elemento 4
indice_elemento 5
indice_elemento 6
indice_elemento 7
indice_elemento 8
indice_elemento 9


/Users/Azeem/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


In [69]:
criterio_fase_solida

0

In [23]:
M_E_A[:,4]

array([ 1.        ,  1.        ,  0.73105845,  1.        ,  1.        ,
        1.        ,  1.        ,  0.63413551,  1.        ,  1.        ])

In [24]:
M_E_A[:,4]

array([ 0.57444247,  0.70056702,  0.55971361,  0.5986876 ,  0.71094938,
        0.55230787,  0.59065322,  0.6532448 ,  1.        ,  0.72111505])

In [30]:
M_E_A

array([[  0.72647829,   0.27352171,  14.        ,   0.74193548,
          0.53364471,   0.32705651,   6.        ,   0.        ],
       [  0.70682695,   0.29317305,  20.        ,   0.59375   ,
          0.54200599,   0.43795191,   7.        ,   0.        ],
       [  0.74280264,   0.25719736,  18.        ,   0.65625   ,
          0.53802167,   0.39215921,   5.        ,   0.        ],
       [  0.7460409 ,   0.2539591 ,  25.        ,   0.38709677,
          0.5642265 ,   0.51368773,   6.        ,   0.        ],
       [  0.74315752,   0.25684248,  18.        ,   0.56666667,
          0.54400348,   0.39205274,   5.        ,   0.        ],
       [  0.75571131,   0.24428869,  23.        ,   0.7027027 ,
          0.53551698,   0.47578661,   5.        ,   0.        ],
       [  0.75894956,   0.24105044,  23.        ,   0.54545455,
          0.54570536,   0.47481513,   5.        ,   0.        ],
       [  0.7513197 ,   0.2486803 ,  17.        ,   0.72727273,
          0.53432092,   0.3721040

In [28]:
def fase_solida(poblacion,mejor_pob_contraria,matriz_resultados):
    """poblacion=poblacion de aniones/cationes
    mejor_pob_contraia=mejor elemento de la poblacion contraria a partir de la cual mutar
    fase=liquida,solida"""
    #probando la fase líquida
    for indice_elemento in list(range(0,(poblacion.shape[0]))):
        
        print("\n")
        print("Me encuentro trabajando en el elmento %s de %s totales" %(indice_elemento,poblacion.shape[0]))
        respaldo_temporal_elemento=np.array(poblacion[indice_elemento])
        print("Elemento:")
        print(respaldo_temporal_elemento)
        
        
        #Encuentra el total de dimensiones M posibles a mutar POR ELEMENTO
        M=M_dimensiones_a_mutar(matriz_resultados[indice_elemento,1],i_actual,i_max)
        
        print("Tiene una probabilidad de error de: %s" % matriz_resultados[indice_elemento,1])
        print("Para el elemento %s se mutan un total máximo de M=%s dimensiones" %(indice_elemento,M))
        
        #Encuentra el total de dimensinoes posibles a mutar, dependiendo del elemento (distancia,fuerza)
        #y que ademas depende de M
        #posicion_temp=cantidad de dimensinoes a mutar por elemento. 
        
        #matriz_resultados[elemento,5]->posicion
        #matriz_resultados[elemnto,4]->fuerza

        
        #-Fase Sólida-#
        
        r=random.random()
        
        if r>0.5:
            print("Separción Drástica")
            
            #separacion drástica
            #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
            posicion_temp=matriz_resultados[indice_elemento,4]*M
            #se redondea para que se un total de dimensiones entero
            posicion_temp=int(np.round_(posicion_temp))
            
        else:
            print("separacion mesurada")
            #Separacion Mesurada
            alfa=random.uniform(.5,1)
            #posicion temporal, maxima cantidad de dimensinoes permitidas a mutar por ese elemento
            posicion_temp=matriz_resultados[indice_elemento,4]*M*alfa
            #se redondea para que se un total de dimensiones entero
            posicion_temp=int(np.round_(posicion_temp))

        #-Fin fase Solida-#
        
        print("Cantidad de dimensiones a mutar específicas para ese elemento: %s" %(posicion_temp))
     
        
        #se actualiza la matriz de resultados
        matriz_resultados[indice_elemento,6]=posicion_temp
        
        #muta unicamente los iguales, aún pese a que haya más elementos posibles a mutar M
        #debido a la posicion_temp
    
        a=np.where((poblacion[indice_elemento]==mejor_pob_contraria)==True)
        #para trabajar sin dificultades se convierte a una lista plana
        b=np.concatenate(a).ravel().tolist()
                
        print("El elemento mejor es: ")
        print(mejor_pob_contraria)
        print("El elemento a evaluar es: ")
        print(poblacion[indice_elemento])
        print("Los indices que son iguales son %s" %(a))
        
        #se seleccionan posicion_temp elementos de forma ALEATORIA, para no elegirse siempre las primeras
        #dimensiones
        
        dimensiones_utilizadas=0
        criterio_fase_solida=0
        
        
        #--------------------------------
        
        if (len(b)!=0):
            if (posicion_temp>len(b)):
                #solución al problema de cuando hay más cantidad de 
                #dimensiones posibles que de elementos, es decir,
                #se toma todo el conjunto de dimensiones iguales
                print("Entre al primer if")
                diferentes_temp_indices=b
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                
                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)
                

            else:
                
                print("Entre al primer else")
                #cuando hay más dimensiones diferentes que dimensiones posibles a mutar
                #se escogen de manera aleatoria
                c=random.sample(b,posicion_temp)
                diferentes_temp_indices=sorted(c)
                diferentes_temp_cantidad=len(diferentes_temp_indices)
                

                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                
            #while para mutar las dimensiones diferentes
            #se resta uno por el while, si son 5, toma 6, debido a que empieza en cero
            while((dimensiones_utilizadas<diferentes_temp_cantidad)):
                print("Entre al primer while")
                print("Diferentes temp cantidad %s" %diferentes_temp_cantidad)
                #muta los diferentea
                #print("\n\n")
                #print(diferentes_temp_indices)
                #print(dimensiones_utilizadas)
                #print(type(diferentes_temp_indices))
                #print(diferentes_temp_indices[0,dimensiones_utilizadas])
                
                print("indice_elemento: %s" %indice_elemento,"iguales temp indices", diferentes_temp_indices,
                     "dimensinoes utilizadas: %s" %dimensiones_utilizadas)

                if (poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] ==0):
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =1
                else:
                    poblacion[indice_elemento,diferentes_temp_indices[dimensiones_utilizadas]] =0


                print("Se muto la dimension: %s" %(diferentes_temp_indices[dimensiones_utilizadas]))
                print("El elemento de la poblacion %s mutado es: ")
                print(poblacion[indice_elemento])

                dimensiones_utilizadas=dimensiones_utilizadas+1      
                
                
        #else:
            #poblacion 
            #poblacion[indice_elemento,7]=1
            #cuando son exactamente iguales el criterio para la fase sólida aumenta
            #criterio_fase_solida=criterio_fase_solida+1
            
    
    return [poblacion,matriz_resultados]
            





In [31]:
[A,M_E_A]=fase_solida(A,mejor_elemento,M_E_A)
#def fase_solida(poblacion,mejor_pob_contraria,matriz_resultados):



Me encuentro trabajando en el elmento 0 de 10 totales
Elemento:
[0 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 0 1 0 0
 1 0 1]
Tiene una probabilidad de error de: 0.247438229162
Para el elemento 0 se mutan un total máximo de M=10.0 dimensiones
Separción Drástica
Cantidad de dimensiones a mutar específicas para ese elemento: 10
El elemento mejor es: 
[1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 0 1 0 1 1 0 1 1
 0 1 0]
El elemento a evaluar es: 
[0 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 0 0 1 0 0
 1 0 1]
Los indices que son iguales son []


Me encuentro trabajando en el elmento 1 de 10 totales
Elemento:
[0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0
 1 1 1]
Tiene una probabilidad de error de: 0.288249123897
Para el elemento 1 se mutan un total máximo de M=12.0 dimensiones
Separción Drástica
Cantidad de dimensiones a mutar específicas para ese elemento: 12
El elemento mejor es: 
[1 0 0 0 0 0 0 1 0 1 1 0

In [32]:
M_E_A

array([[  0.75256177,   0.24743823,  20.        ,   0.        ,
          1.        ,   0.42423147,  10.        ,   0.        ],
       [  0.71175088,   0.28824912,  20.        ,   0.        ,
          1.        ,   0.43647474,  12.        ,   0.        ],
       [  0.71512221,   0.28487779,  18.        ,   0.1       ,
          0.73105845,   0.40046334,   8.        ,   0.        ],
       [  0.73091425,   0.26908575,  20.        ,   0.        ,
          1.        ,   0.43072572,  10.        ,   0.        ],
       [  0.71494477,   0.28505523,  20.        ,   0.        ,
          1.        ,   0.43551657,  11.        ,   0.        ],
       [  0.71822739,   0.28177261,  20.        ,   0.        ,
          1.        ,   0.43453178,   9.        ,   0.        ],
       [  0.73157965,   0.26842035,  20.        ,   0.        ,
          1.        ,   0.43052611,  11.        ,   0.        ],
       [  0.75695338,   0.24304662,  20.        ,   0.18181818,
          0.63413551,   0.4229139